In [2]:
!pip install llama-index-vector-stores-weaviate llama-index llama-index-embeddings-huggingface

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 1.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 1.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.3/368.3 kB 2.2 MB/s eta 0:00:00a 0:00:01
   ━

In [1]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3", device="cuda")

/mnt/annd/.conda/phucnh/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import weaviate
cluster_url = "http://localhost:9090"
client = weaviate.connect_to_local(
    port=9090
)

Thêm summary cho chunk

In [1]:
from langchain_core.prompts import ChatPromptTemplate
import uuid
from langchain.chat_models import ChatOpenAI
import os
from typing import Optional
from langchain_core.pydantic_v1 import BaseModel
from langchain.chains import create_extraction_chain_pydantic

In [2]:
def _get_new_chunk_summary(context):
    PROMPT = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """
                Bạn là người quản lý một nhóm các đoạn văn bản, bảng hoặc biểu đồ, trong đó mỗi đoạn đều đại diện cho một nhóm các câu nói về một chủ đề tương tự nhau.
                Bạn cần tạo ra một tóm tắt ngắn gọn trong 1 câu để thông báo cho người xem biết đoạn văn bản, bảng hoặc biểu đó nói về chủ đề gì.

                Một tóm tắt tốt sẽ nói lên nội dung của đoạn văn bản, bảng hoặc biểu đồ

                Bạn sẽ nhận được một đề xuất mới sẽ được thêm vào đoạn văn bản, bảng hoặc biểu mới. Đoạn văn bản, bảng hoặc biểu đồ mới này cần có một tóm tắt.

                Tóm tắt của bạn nên có tính khái quát tương đối. Nếu bạn nhận được một đề xuất về táo, hãy khái quát nó thành thực phẩm.
                Hoặc nếu là tháng, hãy khái quát nó thành "ngày và thời gian".

                Ví dụ:
                Đầu vào: Đề xuất: Greg thích ăn pizza
                Đầu ra: Đoạn văn bản này chứa thông tin về các loại thực phẩm mà Greg thích ăn.

                Chỉ phản hồi lại bằng tóm tắt của đoạn văn bản mới, không thêm gì khác.
                """,
            ),
            ("user", "Xác định tóm tắt của đoạn văn bản mới mà đề xuất này sẽ được thêm vào:\n{context}"),
        ]
    )
    llm = ChatOpenAI(model='gpt-4o-mini', openai_api_key=os.environ["OPENAI_API_KEY"], temperature=0)

    runnable = PROMPT | llm

    new_chunk_summary = runnable.invoke({
        "context": context
    }).content

    return new_chunk_summary

Cắt Điều trong các bộ luật

In [85]:
import re

In [86]:
def create_Dieu(file_path):
    muc_luc = list()
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    pattern = r'(\* Trang \d+)(.*?)(?=\* Trang \d+|$)'
    article_pattern = r"#+\s+Điều\s+\d+[:.]\s+[^\n]+"
    matches = re.findall(pattern, content, re.DOTALL)
    pages = [match[0] + match[1] for match in matches]
    print(len(pages))
    for i, page in enumerate(pages, 1):
    
        a_matches = re.findall(article_pattern, page)
        for a in a_matches:
            muc_luc.append({"object": a, "page": i})
    

    documents = []
    chapter_pattern = r'(#+\s+Chương\s+[IVXLCDM]+.*?)(?=#+\s+Chương\s+[IVXLCDM]+|$)'


    chapters = re.findall(chapter_pattern, content, re.S)

    if len(chapters) == 0:
        article_pattern = r'(#+\s+Điều\s+\d+[:.].*?)(?=\n+#+\s+Mẫu số|\n+#+\s+Phụ lục|\n+#+\s+Điều\s+\d+|\n+#+\s+Chương\s+[IVXLCDM]+|$)'
        articles = re.findall(article_pattern, content, re.DOTALL)
        for article in articles:
            a_content = re.sub(r'\* Trang \d+', '', article)
            a_content = re.sub(r'[^\w\s.,;:|]', '', a_content)
            a_content = re.sub(r'\n+', ' ', a_content).strip()
            for muc in muc_luc:
                if article.find(muc["object"]) != -1:
                    documents.append({"raw_text": article, "content": a_content, "summary": _get_new_chunk_summary(a_content), "src": file_path.replace(".md", ".pdf"), "chapter": "none", "page": muc["page"]})
                    break
    else:
        article_pattern = r'(#+\s+Điều\s+\d+[:.].*?)(?=\n+#+\s+Mẫu số|\n+#+\s+Phụ lục|\n+#+\s+Điều\s+\d+|\n+#+\s+Chương\s+[IVXLCDM]+|$)'
        chapter_to_articles = {}
        for chapter in chapters:
            chapter_title_match = re.search(r'##\s+(Chương\s+\w+)', chapter)
            if chapter_title_match:
                chapter_title = chapter_title_match.group(1)
                chapter_to_articles[chapter_title] = []
                
                articles = re.findall(article_pattern, chapter, re.S)
                for article in articles:
                    a_content = re.sub(r'\* Trang \d+', '', article)
                    a_content = re.sub(r'[^\w\s.,;:|]', '', a_content)
                    a_content = re.sub(r'\n+', ' ', a_content).strip()
                    for muc in muc_luc:
                        if article.find(muc["object"]) != -1:
                            documents.append({"raw_text": article, "content": a_content, "summary": _get_new_chunk_summary(a_content), "src": file_path.replace(".md", ".pdf"), "chapter": chapter_title, "page": muc["page"]})
                            break
    return documents
            

In [87]:
documents = create_Dieu("parsed_file/14_nghị_định_42_2023_hướng_dẫn_điều_chỉnh_lương_hưu,_trợ_cấp_BHXH.md")

6


In [88]:
len(documents)

6

In [89]:
documents[2]

{'raw_text': '### Điều 3. Nguồn kinh phí thực hiện\n\nNguồn kinh phí thực hiện điều chỉnh lương hưu, trợ cấp bảo hiểm xã hội và trợ cấp hằng tháng áp dụng đối với các đối tượng quy định tại ## Điều 1 Nghị định này được quy định như sau:\n\n1. Ngân sách nhà nước bảo đảm đối với các đối tượng hưởng chế độ bảo hiểm xã hội từ ngân sách nhà nước ngày 01 tháng 01 năm 1995; hướng trợ cấp hằng tháng theo Quyết định số 91/2000/QĐ-TTg ngày 04 tháng 8 năm 2000 của Thủ tướng Chính phủ về việc trợ cấp cho những người đã hết tuổi lao động tại ## điểm ngày 06 tháng 5 năm 2010 của Thủ tướng Chính phủ về việc trợ cấp hằng tháng cho những người có từ đủ 15 năm đến dưới 20 năm công tác thì đã hết thời hạn hưởng trợ cấp mất sức lao động vào các đối tượng quy định tại ## điểm d, ## điểm e và ## điểm g khoản 1 ## Điều 1 Nghị định này; hướng lương hưu theo Nghị định số 159/2006/NĐ-CP ngày 28 tháng 12 năm 2006 của Chính phủ về việc thực hiện chế độ hưu trí đối với quân nhân trực tiếp tham gia kháng chiến chốn

Cắt bảng

In [3]:
def create_Form(file_path):
    muc_luc = list()
    documents = []
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    pattern = r'(\* Trang \d+)(.*?)(?=\* Trang \d+|$)'
    form_pattern = r"#+\s+.*Mẫu số.*\s+\d+[^\n]+"
    table_pattern = r'\*Begin table\*.*?\*End table\*'
    matches = re.findall(pattern, content, re.DOTALL)

    pages = [match[0] + match[1] for match in matches]

    cur_form = " "
    for i, page in enumerate(pages, 1):
        f_matches = re.findall(form_pattern, page)
        for f in f_matches:

            cur_form = f
            muc_luc.append({"object": f, "page": i})

        tables = re.findall(table_pattern, page, re.DOTALL)
        for table in tables:
            # print(table)
            t_content = re.sub(r'\* Trang \d+', '', table)
            t_content = re.sub(r'\*Begin table\*', ' ', t_content).strip()
            t_content = re.sub(r'\*End table\*', ' ', t_content).strip()
            documents.append({"raw_text": table, "content": t_content, "summary": _get_new_chunk_summary(t_content), "src": file_path.replace(".md", ".pdf"), "chapter": cur_form, "page": i})


    form_pattern = r"(#\s*\**Mẫu số \d+/PL[IVXLCDM]+\**\n.*?)(?=#\s*\**Mẫu số|\Z)"
    forms = re.findall(form_pattern, content, re.DOTALL)
    for form in forms:
        # print(form)
        f_content = re.sub(r'\* Trang \d+', '', form)
        f_content = re.sub(r'[^\w\s.,;:|]', '', f_content)
        f_content = re.sub(r'\n+', ' ', f_content).strip()
        for muc in muc_luc:
            if form.find(muc["object"]) != -1:
                page_num = muc["page"]
                documents.append({"raw_text": form, "content": f_content, "summary": _get_new_chunk_summary(f_content), "src": file_path.replace(".md", ".pdf"), "chapter": "none", "page": page_num})
                break
    return documents

    

In [97]:
documents = create_Form("parsed_file/8_Nghị_định_15_2020_về_lao_động_nước_ngoài_tại_VN.md")

In [11]:
with open("parsed_file/14_nghị_định_42_2023_hướng_dẫn_điều_chỉnh_lương_hưu,_trợ_cấp_BHXH.md", 'r', encoding='utf-8') as file:
    content = file.read()

import re
pattern = r'Trang 1(.*?)(?=\* Trang 2)'
match = re.findall(pattern, content, re.DOTALL)[0]
pattern__1 = [r'(BỘ LUẬT\s*\n)(.*)', r'(LUẬT\s*\n)(.*)', r'(NGHỊ ĐỊNH\s*\n)(.*)']
text = re.findall(r'(NGHỊ ĐỊNH\s*\n)(.*)', match)
# for p in pattern__1:
#     text = re.findall(p, match)
#     break
print(text)
ten_luat = text[0][0].strip() + " " + text[0][1].strip()
print(ten_luat)

[('NGHỊ ĐỊNH  \n', 'Điều chỉnh lương hưu, trợ cấp bảo hiểm xã hội và trợ cấp hằng tháng  ')]
NGHỊ ĐỊNH Điều chỉnh lương hưu, trợ cấp bảo hiểm xã hội và trợ cấp hằng tháng


In [12]:
import json
with open("parsed_file/14_nghị_định_42_2023_hướng_dẫn_điều_chỉnh_lương_hưu,_trợ_cấp_BHXH.json", 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)
for d in data:
    d["reference"] = ten_luat

with open('parsed_file/14_nghị_định_42_2023_hướng_dẫn_điều_chỉnh_lương_hưu,_trợ_cấp_BHXH.json', 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

In [15]:
raw = [
    """| Chỉ tiêu                                                                                                                                      | Đầu kỳ | Tăng trong kỳ | Giảm trong kỳ | Cuối kỳ | Số lao động tham gia bảo hiểm bắt buộc (người) | Ghi chú |
|-----------------------------------------------------------------------------------------------------------------------------------------------|--------|---------------|---------------|---------|------------------------------------------------|---------|
| 1. Tổng số lao động theo hợp đồng lao động của doanh nghiệp, trong đó:                                                                        |        |               |               |         |                                                |         |
| a) Số lao động làm việc tại doanh nghiệp cho thuê lại, chia ra:                                                                     |        |               |               |         |                                                |         |
| - Số lao động có hợp đồng lao động không xác định thời hạn                                                                                   |        |               |               |         |                                                |         |
| - Số lao động có hợp đồng lao động xác định thời hạn                                                                                         |        |               |               |         |                                                |         |
| b) Số lao động cho thuê lại, chia ra:                                                                                                        |        |               |               |         |                                                |         |
| - Thời hạn cho thuê lại dưới 03 tháng                                                                                                        |        |               |               |         |                                                |         |
| - Thời hạn cho thuê lại từ 03 tháng đến dưới 06 tháng                                                                                        |        |               |               |         |                                                |         |
| - Thời hạn cho thuê lại từ 06 tháng đến 12 tháng                                                                                             |        |               |               |         |                                                |         |
| 2. Số lao động đang cho thuê lại của doanh nghiệp                                                                                            |        |               |               |         |                                                |         |
| - Trong địa bàn tỉnh                                                                                                                         |        |               |               |         |                                                |         |
| - Ngoài địa bàn tỉnh                                                                                                                         |        |               |               |         |                                                |         |""",
    """| TT | Công việc cho thuê lại | Số doanh nghiệp được cấp giấy phép cho thuê lại lao động (1) | Số lao động cho thuê lại (người) | Số lao động theo thời hạn cho thuê lại lao động (người) | Các chế độ của người lao động cho thuê lại | Số lao động thuê lại tham gia bảo hiểm bắt buộc (người) (7) | Ghi chú |
|----|-----------------------|--------------------------------------|-----------------------------|------------------------------------------|-------------------------------|---------------------------------|--------|
|    |                       | Trong địa bàn tỉnh | Ngoài địa bàn tỉnh | Của doanh nghiệp trong địa bàn tỉnh | Của doanh nghiệp ngoài địa bàn tỉnh | Dưới 6 tháng | Từ 6 - 12 tháng | Khác | Tiền lương bình quân (đồng/người/tháng) | Thu nhập bình quân (đồng/người/tháng) | Chế độ phúc lợi |                                  |        |
| 1  |                       | 2                                | 3                              | 4                                                      | 5                                                       | 6                | 7                | 8    | 9                                          | 10                | 11                        | 12                                    | 13             | 14     |
| 1  |                       |                                    |                               |                                                        |                                                         |                  |                  |      |                                            |                    |                            |                                        |                |        |
| 2  |                       |                                    |                               |                                                        |                                                         |                  |                  |      |                                            |                    |                            |                                        |                |        |""",
    """Appendix 3: EVPC - Hazardous Work and Special Hazardous Works

| Mechanical, Metallurgy (Reference number) | Occupation or work                                                                        | Working conditions                                                                                                   | Working conditions class | Applies in EVPC practical conditions       |
|-------------------------------------------|-------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------|--------------------------|-------------------------------------------|
|                                           |                                                                                           |                                                                                                                      |                          | Main occupational (*1)                       | Concurrent (*2)       |
| 31                                        | Metal Melting                                                                             | Công việc nặng nhọc, nguy hiểm, chịu tác động của nóng và hơi khí độc.<br>Under the impact of heat and toxic gases. | V                        | Nhóm nấu luyện<br>Melting Group             |                         |
| 40                                        | Foundry work and rolling, metal shaping                                                   | Công việc nặng nhọc, rất nguy hiểm, chịu tác động của nóng, bụi, hơi khí độc.<br>Under the impact of heat, toxic gases.| V                        | Nhóm Hoàn thiện<br>Molding Group            |                         |
| 72                                        | Cleaning cast works                                                                       | Làm việc trong nhà đúc, chịu tác động của nóng, rung, tiếng ồn, bụi độc.<br>Heat, noise and dust.                     | IV                       | Nhóm Lắp ráp AT2<br>Assembly AT2 (sandblasting)<br>Nhóm hoàn thiện Fetling Group |                         |
| 80                                        | Spraying paint, method of thermal hot melting                                             | Tiếp xúc với hóa chất độc hại, thường xuyên tiếp xúc với nước có nhiệt độ cao.<br>Exposed to thermal hot chemicals often.| IV                       | Nhóm Lắp ráp AT2<br>Assembly Group/ AT2     |                         |
| 53                                        | Metal Casting                                                                             | Tiếp xúc với nhôm lỏng và kim loại, rung và ồn.<br>Heat, metal dust, vibration and noise.                             | IV                       | Lắp ráp tại AT3<br>Assembly AT3 (W. GR)    |                         |
| 98                                        | Welding, Steam welding                                                                    | Chịu tác động của hơi khí độc.<br>Exposed to toxic gases.                                                             | IV                       | Nhóm gò hàn ren<br>Fabrication Group       |                         |
| 61                                        | Operate metal punching machines                                                           | Heat, noise, vibration, visual stress.                                                                               | IV                       | Fabrication<br>(Thanh & Cuong)             |                         |
| 16                                        | Operating forklifts for store (not containers)                                            | Môi trường bụi, ồn, cường độ lao động khẩn trương<br>Working in dusty, noisy environment.                            | IV                       | Ware house team excluding Ms. Dung & Ms. Loan |                         |
| 62                                        | Operating cranes in casting factory                                                       | Nóng, bụi, căng thẳng thần kinh, tâm lý<br>Heat, dust, noxious gases                                                 | IV                       | Nhóm vận hành cẩu trục (Kho)<br>Crane team of Warehouse |                         |
| 118                                       | Burning and operating other annealing furnaces                                            | Công việc nặng nhọc, thường xuyên chịu tác động của nhiệt độ cao, CO và CO2.<br>Handling heat and gases.             | IV                       | Support by<br>Fettling Group               |                         |
| 131                                       | Drilling, tapping and turning cast iron                                                   | Bụi, căng thẳng thị giác, tập trung chú ý cao<br>Dust, visual stress, high concentration of attention..              | IV                       | Machining Group                            |                         |
| 23                                        | Working with industrial machines while performing mechanical therapy in PMT, PT and XRT in MT | Tiếp xúc với hóa chất độc hại, chịu tác động của cực tím, tia bức xạ<br>Contact with toxic substances, UV, radiation. | IV                       | Inspection Group                           |                         |

Noted:
1. The name of the occupation will be recorded in the Social insurance book and apply the annual paid leave regime for staff working under Hazardous conditions.
2. Apply the same regime as employees working in normal conditions.""",
    """| **Chỉ tiêu**                                                            | **Số doanh nghiệp** | **Loại hình doanh nghiệp** |                          |                                |                                | **Tổng số lao động sử dụng** |                                 |                              | **Số lượng doanh nghiệp thuê lại lao động** |                    |                       | **Ghi chú** |
|:----------------------------------------------------------------------|:-------------------:|:--------------------------:|:------------------------:|:------------------------------:|:------------------------------:|:-----------------------------:|:-------------------------------:|:----------------------------:|:------------------------------------------:|:------------------:|:---------------------:|:---------:|
|                                                                       |                     | **Doanh nghiệp nhà nước**  | **Doanh nghiệp tư nhân** | **Doanh nghiệp có vốn đầu tư nước ngoài** |                                | **Số lao động làm việc tại doanh nghiệp cho thuê lại** | **Số lao động cho thuê lại (người)** |                              | **Trong địa bàn tỉnh**                         | **Ngoài địa bàn tỉnh** |                       |           |
| 1. Số doanh nghiệp đã được cấp phép đầu kỳ báo cáo                    |                     |                            |                          |                                  |                                |                               |                                 |                              |                                            |                    |                       |           |
| 2. Số doanh nghiệp được cấp giấy phép lần đầu trong kỳ báo cáo       |                     |                            |                          |                                  |                                |                               |                                 |                              |                                            |                    |                       |           |
| 3. Số doanh nghiệp được gia hạn giấy phép                            |                     |                            |                          |                                  |                                |                               |                                 |                              |                                            |                    |                       |           |
| 4. Số doanh nghiệp cấp lại giấy phép                                 |                     |                            |                          |                                  |                                |                               |                                 |                              |                                            |                    |                       |           |
| 5. Tổng số doanh nghiệp giảm trong kỳ báo cáo, trong đó:             |                     |                            |                          |                                  |                                |                               |                                 |                              |                                            |                    |                       |           |
| - Doanh nghiệp hết hạn giấy phép mà không được gia hạn, cấp lại       |                     |                            |                          |                                  |                                |                               |                                 |                              |                                            |                    |                       |           |
| - Doanh nghiệp bị thu hồi giấy phép                                  |                     |                            |                          |                                  |                                |                               |                                 |                              |                                            |                    |                       |           |
| 6. Số doanh nghiệp có giấy phép đang hoạt động cuối kỳ               |                     |                            |                          |                                  |                                |                               |                                 |                              |                                            |                    |                       |           |""",
    """| STT | Họ tên | Mã số BHXH | Ngày tháng năm sinh | Giới tính | Số CCCD/CMND/Hộ chiếu | Cấp bậc, chức vụ, chức danh nghề, nơi làm việc | Nhà quản lý | Chuyên môn kỹ thuật bậc cao | Chuyên môn kỹ thuật bậc trung | Khác | Hệ số/Bậc lương | Chức vụ | Thâm niên VK | Thu hút | Lưu động | Ngành nghề | Khu vực | Khoán | Phụ cấp lương bổ sung | Ngành/nghề nặng nhọc, độc hại | Ngày bắt đầu HĐLD xác định thời hạn | Ngày kết thúc HĐLD xác định thời hạn | Ngày bắt đầu HĐLD không xác định thời hạn | Ngày kết thúc HĐLD không xác định thời hạn | Hiệu lực HĐLD khác (dưới 1 tháng, thử việc) | Thời điểm đơn vị bắt đầu đóng BHXH | Thời điểm đơn vị kết thúc đóng BHXH | Ghi chú |
|-----|--------|-----------|---------------------|-----------|-----------------------|---------------------------------------------|-------------|--------------------------|--------------------------|------|-----------------|--------|------------|--------|----------|------------|---------|--------|-----------------------|-----------------------------------|---------------------------------|-------------------------------|--------------------------------------|-------------------------------------|--------------------------------------|---------------------------------|---------------|
| 1   |        |           |                     |           |                       |                                             |             |                          |                          |      |                 |        |            |        |          |            |         |        |                       |                                   |                                 |                               |                                      |                                     |                                      |                                 |               |
| ... |        |           |                     |           |                       |                                             |             |                          |                          |      |                 |        |            |        |          |            |         |        |                       |                                   |                                 |                               |                                      |                                     |                                      |                                 |               |
| Tổng|        |           |                     |           |                       |                                             |             |                          |                          |      |                 |        |            |        |          |            |         |        |                       |                                   |                                 |                               |                                      |                                     |                                      |                                 |               |""",
    """| STT | Người sử dụng lao động | Tổng | Lao động nữ | Lao động trên 35 tuổi | Số lao động tham gia BHXH bắt buộc | Nhà quản lý | Chuyên môn kỹ thuật bậc cao | Chuyên môn kỹ thuật bậc trung | Khác | Số lao động tham gia HĐLĐ không xác định thời hạn | Số lao động tham gia HĐLĐ xác định thời hạn | Số lao động tham gia HĐLĐ khác (dưới 1 tháng, thử việc) | Ghi chú |
|-----|------------------------|------|-------------|-----------------------|-----------------------------------|-------------|---------------------------|---------------------------|-------|-------------------------------------------|----------------------------------------|----------------------------------------------|---------|
| 1   | Doanh nghiệp           |      |             |                       |                                   |             |                           |                           |       |                                           |                                        |                                              |         |
| 2   | Hợp tác xã             |      |             |                       |                                   |             |                           |                           |       |                                           |                                        |                                              |         |
| 3   | Cơ quan, tổ chức       |      |             |                       |                                   |             |                           |                           |       |                                           |                                        |                                              |         |
| Tổng|                        |      |             |                       |                                   |             |                           |                           |       |                                           |                                        |                                              |         |

Ghi chú:
- (1): Bộ Lao động - Thương binh và Xã hội.
- (2): Vị trí việc làm phân loại theo:
  - Cột (7) Nhà quản lý: Nhóm này bao gồm những nhà lãnh đạo, quản lý làm việc trong các ngành, các cấp và trong các cơ quan, tổ chức, doanh nghiệp có giữ các chức vụ, có quyền quản lý, chỉ huy, điều hành từ trung ương tới cấp xã;
  - Cột (8) Chuyên môn kỹ thuật bậc cao: Nhóm này bao gồm những nghề đòi hỏi phải có kiến thức chuyên môn, nghiệp vụ và kinh nghiệm ở trình độ cao (đại học trở lên) trong lĩnh vực khoa học và kỹ thuật, sức khỏe, giáo dục, kinh doanh và quản lý, công nghệ thông tin và truyền thông, luật pháp, văn hóa, xã hội;
  - Cột (9) Chuyên môn kỹ thuật bậc trung: Nhóm này bao gồm những nghề đòi hỏi kiến thức và kinh nghiệm ở trình độ bậc trung (cao đẳng, trung cấp) về các lĩnh vực khoa học và kỹ thuật, sức khỏe, kinh doanh và quản lý, luật pháp, văn hóa, xã hội, thông tin và truyền thông, giáo viên, giáo dục, công nghệ thông tin.""",
  """VĂN BẢN THỎA THUẬN LÀM THÊM GIỜ

- Thời gian làm thêm: Kể từ ngày ........ đến ngày ...... tháng .... năm ......
- Địa điểm làm thêm: ...............................................................
- Lý do làm thêm: .................................................................

| STT | Họ tên | Nghề, công việc đang làm (2) | Số giờ làm việc trong ngày (2) | Số giờ làm thêm (trong ngày, tuần, ...) (3) | Chữ ký của người lao động |
|-----|--------|------------------------------|------------------------------|------------------------------------------|----------------------------------|
| 1   |        |                              |                              |                                          |                                  |
| 2   |        |                              |                              |                                          |                                  |
| 3   |        |                              |                              |                                          |                                  |
| 4   |        |                              |                              |                                          |                                  |

### Ghi chú:
1. Mẫu này lập khi ký văn bản với nhiều người lao động; trường hợp ký riêng từng người lao động thì điều chỉnh các thông tin tương ứng.
2. Trường hợp đã sử dụng bảng chấm công và công việc, giờ làm việc không thay đổi trong nhiều ngày, nhiều tháng đã ghi trong bảng chấm công thì không bắt buộc có các cột này trong bản thỏa thuận.
3. Có thể ghi thỏa thuận riêng theo từng ngày hoặc theo tuần, theo tháng hoặc thỏa thuận kết hợp nhiều nội dung về thời giờ làm thêm.""",
    """I. TÌNH HÌNH TUYỂN DỤNG NGƯỜI LAO ĐỘNG VIỆT NAM TRONG NĂM...

| STT | Họ và tên | Năm sinh | Trình độ chuyên môn kỹ thuật cao nhất | Vị trí việc làm | Tuyển trực tiếp | Tổ chức dịch vụ việc làm | Doanh nghiệp cho thuê lại lao động | Tuyển thông qua tổ chức có thẩm quyền tuyển, quản lý người lao động Việt Nam |
|-----|-----------|----------|--------------------------------------|-----------------|----------------|--------------------------|---------------------------------|--------------------------------------------------------------------------------|
| 1   |           |          |                                      |                 |                |                          |                                 |                                                                                |
| 2   |           |          |                                      |                 |                |                          |                                 |                                                                                |"""
]


# page = [5, 5, 6, 6, ""]
import json
with open("parsed_file/law_2. Tai lieu Khach hang cung cap_5. Salary Regulation 2024 (V).json", 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)
for i in range(15, -1):
    data[i]["content"] = data[i]["content"].strip()

with open('parsed_file/law_2. Tai lieu Khach hang cung cap_5. Salary Regulation 2024 (V).json', 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

In [98]:
len(documents)

27

In [100]:
documents[19]

{'raw_text': '# **Mẫu số 02/PLI**\n\n**TÊN DOANH NGHIỆP/TỔ CHỨC**  \nCỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM  \nĐộc lập - Tự do - Hạnh phúc  \nSố: .........  \nV/v giải trình thay đổi nhu cầu sử dụng người lao động nước ngoài  \n.........., ngày....tháng....năm.....  \n\nKính gửi: .............. (1) ..............  \n\nCăn cứ vào văn bản số... (ngày/tháng/năm) của... (2) về việc thông báo chấp thuận vị trí công việc sử dụng người lao động nước ngoài và nhu cầu thực tế của (tên doanh nghiệp/tổ chức).  \n\nThông tin về doanh nghiệp/tổ chức: tên doanh nghiệp/tổ chức, loại hình doanh nghiệp/tổ chức, tổng số lao động đang làm việc tại doanh nghiệp/tổ chức, trong đó số lao động nước ngoài đang làm việc, địa chỉ, điện thoại, fax, email, website, thời hạn của giấy phép kinh doanh/hoạt động, lĩnh vực kinh doanh/hoạt động, người nộp hồ sơ của doanh nghiệp/tổ chức có liên hệ khi cần thiết (số điện thoại, email).  \n\nBáo cáo giải trình thay đổi nhu cầu sử dụng người lao động nước ngoài như sau:  \n\n1

In [2]:
from llama_index.embeddings.text_embeddings_inference import TextEmbeddingsInference

TEI_URL="http://localhost:5555"
MODEL_NAME = "bkai/bge-m3"
embed_model = TextEmbeddingsInference(model_name=MODEL_NAME,
                                      base_url=TEI_URL)

In [1]:
from importlib.metadata import version, PackageNotFoundError

In [2]:
__version__ = version("weaviate-client")
__version__

'3.26.6'

In [1]:
import weaviate
cluster_url = "http://localhost:9090"
client = weaviate.Client(cluster_url)

/mnt/annd/.conda/annd/lib/python3.10/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.6. The latest version is 4.8.0.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [5]:
abc = client.schema.get(class_name="DocumentLaw")


In [ ]:
client.batch.delete_objects()

In [5]:
client.schema.delete_class('DocumentLaw')

In [7]:
from llama_index.vector_stores.weaviate import WeaviateVectorStore

In [8]:
vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name="DocumentLaw"
)

In [9]:
from llama_index.core import StorageContext, VectorStoreIndex
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [10]:
import os
import json

def merge_json(folder_path):
    merged_data = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding="utf-8") as json_file:
                data = json.load(json_file)
                if isinstance(data, list):
                    merged_data.extend(data)

    return merged_data

folder_path = "parsed_file"
merged_array = merge_json(folder_path=folder_path)


In [12]:
merged_array[1]

{'raw_text': '### Điều 2. Đối tượng áp dụng  \n1. Người lao động, người sử dụng lao động thuộc đối tượng bắt buộc tham gia bảo hiểm thất nghiệp theo quy định tại Điều 43 Luật Việc làm.  \n2. Người lao động là người quản lý doanh nghiệp, quản lý hợp tác xã hưởng tiền lương và có tham gia bảo hiểm xã hội bắt buộc theo quy định của pháp luật về bảo hiểm xã hội.  \n3. Tổ chức bảo hiểm xã hội, Sở Lao động - Thương bình và Xã hội tỉnh, thành phố trực thuộc Trung ương (sau đây viết tắt là Sở Lao động - Thương bình và Xã hội), trung tâm dịch vụ việc làm do cơ quan quản lý nhà nước về việc làm thành lập được giao các nhiệm vụ theo quy định tại Khoản 2 Điều 38 Luật Việc làm (sau đây viết tắt là trung tâm dịch vụ việc làm).  \n4. Cơ quan, tổ chức, cá nhân có liên quan đến bảo hiểm thất nghiệp theo quy định của Luật Việc làm.  \n',
 'content': ' Điều 2. Đối tượng áp dụng  \n1. Người lao động, người sử dụng lao động thuộc đối tượng bắt buộc tham gia bảo hiểm thất nghiệp theo quy định tại Điều 43 Lu

In [1]:
import os
import json
import re

def merge_json(folder_path):
    merged_data = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            new_data = []
            with open(file_path, 'r', encoding="utf-8") as json_file:
                data = json.load(json_file)
                for chunk in data:
                    chunk["raw_text"] = re.sub(r'\* Trang \d+', '', chunk["raw_text"])
                    chunk["raw_text"] = re.sub(r'\*Begin table\*', ' ', chunk["raw_text"])
                    chunk["raw_text"] = re.sub(r'\*End table\*', ' ', chunk["raw_text"])
                    chunk["raw_text"] = re.sub(r'\n+', '\n', chunk["raw_text"])
                    markdown_special_chars = r'[\\#\*]'
                    chunk["content"] = re.sub(markdown_special_chars, '', chunk["raw_text"])
                    new_data.append(chunk)
            with open(file_path, 'w', encoding="utf-8") as json_file:
                json.dump(new_data, json_file, ensure_ascii=False, indent=4)

    return merged_data

folder_path = "parsed_file"
merged_array = merge_json(folder_path=folder_path)


In [15]:
len(merged_array)

508

In [19]:
merged_array[2]

{'raw_text': '### Điều 3. Điều kiện được hỗ trợ\nNgười sử dụng lao động được hỗ trợ kinh phí đào tạo, bồi dưỡng, nâng cao trình độ kỹ năng nghề để duy trì việc làm cho người lao động quy định tại Khoản 1 Điều 47 Luật Việc làm khi có đủ các điều kiện sau:\n1. Đồng đú bảo hiểm thất nghiệp theo quy định tại Khoản 2 Điều 44 Luật Việc làm liên tục từ đủ 12 tháng trở lên tính đến tháng liền trước của tháng đề nghị hỗ trợ kinh phí đào tạo, bồi dưỡng, nâng cao trình độ kỹ năng nghề để duy trì việc làm cho người lao động nếu người sử dụng lao động đã đóng bảo hiểm thất nghiệp của tháng đó.\n2. Gặp khó khăn do suy giảm kinh tế hoặc vì lý do bất khả kháng thuộc phải đãi đôì cơ cấu hoặc công nghệ sản xuất, kinh doanh và dẫn đến nguy cơ phải cắt giảm số lao động hiện có từ 30% hoặc từ 50 lao động trở xuống với 100 lao động trở lên đối với người sử dụng lao động có sử dụng trên 300 lao động, không kể lao động giao kết hợp đồng lao động vơi thời hạn dưới 03 tháng.\nNhững trường hợp được coi là bất kh

In [13]:
import copy
from llama_index.core.schema import TextNode

all_nodes = []
excluded_embed_metadata_keys=["raw_text", "summary", "src", "chapter", "page", "reference"]
excluded_llm_metadata_keys=["raw_text", "summary", "src", "chapter", "page", "reference"]

for chunk in merged_array:
    chunk_text = chunk["content"]
    metadata = copy.deepcopy(chunk)
    metadata.pop("content")
    reference=chunk.get("reference","")
    if reference:
      reference=f"Nguồn: {reference}\n\n"
    else:
       reference=""
    chapter=chunk.get("chapter","")
    if chapter and chapter!='none':
      chapter=f"Chương: {chapter}\n\n"
    else:
       chapter=""
    node = TextNode(
        text = reference+" "+chapter+" "+chunk_text,
        extra_info= metadata,
        excluded_embed_metadata_keys=excluded_embed_metadata_keys,
        excluded_llm_metadata_keys=excluded_llm_metadata_keys,
    )
    all_nodes.append(node)

In [14]:
index = VectorStoreIndex(all_nodes,
    storage_context=storage_context,
    embed_model=embed_model,
    insert_batch_size=100,
    show_progress=True)

Generating embeddings:   0%|          | 0/100 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/100 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/100 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/100 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/100 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/8 [00:00<?, ?it/s]